In [ ]:
import os

import pickle
import random
import numpy as np

import torch
from torch.autograd import Variable

In [ ]:
#import argparse
#parser = argparse.ArgumentParser(description='PyTorch Relations-from-Stream sort-of-CLVR Example')
#args = parser.parse_args()

from attrdict import AttrDict
args = AttrDict()

In [ ]:
args.batch_size = 32
args.cuda = True
args.lr   = 0.0001
args.seed = 5

In [ ]:
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [ ]:
# Load in a batch of images to test dimensions, etc...
bs = args.batch_size

input_img = torch.FloatTensor(bs, 3, 75, 75)
input_qst = torch.FloatTensor(bs, 11)
label = torch.LongTensor(bs)

if args.cuda:
    input_img = input_img.cuda()
    input_qst = input_qst.cuda()
    label = label.cuda()
    
input_img = Variable(input_img)
input_qst = Variable(input_qst)
label = Variable(label)

In [ ]:
data_dirs = './data'

filename = os.path.join(data_dirs, 'sort-of-clevr.pickle')
with open(filename, 'rb') as f:
  train_datasets, test_datasets = pickle.load(f)

def cvt_data_axis(data):
    img = [e[0] for e in data]
    qst = [e[1] for e in data]
    ans = [e[2] for e in data]
    return (img,qst,ans)

rel_train = []
#norel_train = []

for img, relations, norelations in train_datasets:
    img = np.swapaxes(img,0,2)
    for qst,ans in zip(relations[0], relations[1]):
        rel_train.append((img,qst,ans))
    #for qst,ans in zip(norelations[0], norelations[1]):
    #    norel_train.append((img,qst,ans))

rel = cvt_data_axis(rel_train)
#norel = cvt_data_axis(norel_train)

In [ ]:
def tensor_data(data, i):
    img = torch.from_numpy(np.asarray(data[0][bs*i:bs*(i+1)]))
    qst = torch.from_numpy(np.asarray(data[1][bs*i:bs*(i+1)]))
    ans = torch.from_numpy(np.asarray(data[2][bs*i:bs*(i+1)]))

    input_img.data.resize_(img.size()).copy_(img)
    input_qst.data.resize_(qst.size()).copy_(qst)
    label.data.resize_(ans.size()).copy_(ans)

tensor_data(rel, 0)    # Loads batch 0 into input_img, input_qst amd label
#tensor_data(norel, 0)

In [ ]:
#from model import RN, CNN_MLP, RFS
import model

import importlib
importlib.reload(model)

m = model.RFS(args)
if args.cuda:
    m.cuda()
m.train();

#accuracy_rel = m.train_(input_img, input_qst, label)
#accuracy_norel = m.train_(input_img, input_qst, label)

m.optimizer.zero_grad()
output = m(input_img, input_qst)

In [ ]:
a = torch.zeros(8, 2)
b = torch.zeros(8, 7)

c = torch.cat( (a,b), 1)
c

In [ ]:
p = torch.rand( (1,5) )
p
p.expand( (6,5) )